In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
import gensim

### пункт 0

Загрузите датасет с помощью функции fetch_20newsgroups

In [0]:
d = fetch_20newsgroups(remove=('headers', 'footer', 'quotes')).data

### part 1: LDA из sklearn

- Однопоточный, неэффективный.
- Совместим в векторайзерами sklearn
- легко исопльзовать

### пункт 1

1) создайте объект класса CountVectorizer 

2) примените его к вашей коллекции, причем так, чтобы коллекция документов была представлена не всеми словами из словаря, а только 1000 словами, которые наиболее часто встречаются в коллекции

(для этого надо воспользоваться методом fit_transform и указать соответствующий параметр)

In [0]:
cv = CountVectorizer(stop_words='english')

transformed_dataset = cv.fit_transform(d)

### пункт 2
Получите названия тех топ-1000 слов, которые вы только что закодировали векторайзером

In [0]:
import numpy as np

inv_vocab = dict([(val, key) for key, val in cv.vocabulary_.items()])
counter = np.array(transformed_dataset.sum(axis=0))[0]
top_indices = np.argsort(counter, )[-1000:]
tf_features = transformed_dataset[:,top_indices]
tf_feature_names = [inv_vocab[i] for i in top_indices]
assert len(tf_feature_names) == 1000

Обучите LatentDirichletAllocation из sklearn со следующими параметрами n_topics=20, max_iter=50, learning_method='batch'

In [0]:
lda = LatentDirichletAllocation(20, max_iter=50, learning_method='batch')
lda.fit(tf_features)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=20, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

Воспользуйтесь следующей функцией, чтобы вывести ключевые слова конкретной темы.

In [0]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print 'Topic {}:'.format(topic_idx)
        print ' '.join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]) 

In [0]:
n_top_words = 10
print_top_words(lda, tf_feature_names, n_top_words)

Topic 0:
don just like think know people good time ve really
Topic 1:
game year team car games play dod season hockey good
Topic 2:
00 price 50 new 000 sale 100 offer 10 20
Topic 3:
available ftp software image server version edu pub graphics faq
Topic 4:
government law state states right power rights people military federal
Topic 5:
said armenian db turkish people armenians went children armenia did
Topic 6:
university research new center 1993 science national information april technology
Topic 7:
10 11 12 16 17 14 25 15 18 13
Topic 8:
people israel evidence jews question does israeli human true argument
Topic 9:
gun control guns group use crime firearms news service self
Topic 10:
god jesus bible believe church christian faith christ christians does
Topic 11:
mr president going stephanopoulos think know don money jobs said
Topic 12:
key encryption chip keys clipper security privacy public use w7
Topic 13:
space nasa __ 0d ___ gov launch _o 145 air
Topic 14:
edu com mail cs uk interne

### part 2: Gensim

- Неэффективный.
- Совместим в векторайзерами sklearn.
- Легко использовать.
- Поддерживает несколько входных форматов.

Преобразуйте корпус следующим образом

In [0]:
corpus = gensim.matutils.Sparse2Corpus(tf_features, documents_columns=False)

### пункт 3
Заполните словарь id2word в виде {индекс:слово}

In [0]:
id2word = dict(enumerate(tf_feature_names))

### пункт 4
Обучите модель LDA из gensim (для этого надо воспользоваться gensim.models.ldamodel.LdaModel)

In [0]:
lda = gensim.models.ldamodel.LdaModel(corpus, num_topics=20, id2word=id2word)

Выведите наиболее вероятные слова для каждой темы

In [0]:
lda.print_topics(lda.num_topics)

[(0,
  u'0.015*"key" + 0.014*"use" + 0.012*"bit" + 0.011*"like" + 0.010*"windows" + 0.010*"dos" + 0.009*"used" + 0.009*"using" + 0.009*"software" + 0.008*"serial"'),
 (1,
  u'0.027*"window" + 0.018*"win" + 0.014*"server" + 0.013*"games" + 0.012*"vs" + 0.012*"package" + 0.012*"run" + 0.012*"ms" + 0.011*"game" + 0.010*"problem"'),
 (2,
  u'0.024*"power" + 0.016*"problem" + 0.013*"dod" + 0.013*"like" + 0.012*"disk" + 0.012*"bike" + 0.011*"ground" + 0.011*"hard" + 0.010*"apple" + 0.010*"use"'),
 (3,
  u'0.018*"people" + 0.017*"said" + 0.016*"gun" + 0.015*"000" + 0.012*"armenian" + 0.012*"turkish" + 0.012*"armenians" + 0.012*"war" + 0.012*"children" + 0.011*"guns"'),
 (4,
  u'0.023*"don" + 0.021*"just" + 0.019*"like" + 0.018*"think" + 0.016*"know" + 0.015*"people" + 0.014*"time" + 0.013*"good" + 0.010*"say" + 0.009*"ll"'),
 (5,
  u'0.138*"ax" + 0.021*"max" + 0.019*"orbit" + 0.012*"like" + 0.009*"know" + 0.008*"just" + 0.007*"edu" + 0.007*"a86" + 0.007*"think" + 0.006*"things"'),
 (6,
  u'0.

### пункт 5
Обучите модель LSI из gensim (для этого надо воспользоваться gensim.models.lsimodel.LsiModel)

In [0]:
lsi = gensim.models.lsimodel.LsiModel(corpus, 20, id2word)

Выведите наиболее вероятные слова для каждой темы

In [0]:
lsi.print_topics(lsi.num_topics)

[(0,
  u'0.997*"ax" + 0.072*"max" + 0.016*"g9v" + 0.012*"b8f" + 0.010*"a86" + 0.009*"pl" + 0.007*"1d9" + 0.006*"1t" + 0.006*"145" + 0.006*"bhj"'),
 (1,
  u'0.387*"145" + 0.378*"a86" + 0.374*"b8f" + 0.359*"g9v" + 0.324*"0d" + 0.226*"1d9" + 0.211*"0t" + 0.187*"2di" + 0.172*"_o" + 0.171*"34u"'),
 (2,
  u'-0.309*"file" + -0.248*"edu" + -0.171*"use" + -0.144*"available" + -0.129*"com" + -0.122*"program" + -0.121*"information" + -0.117*"people" + -0.117*"pub" + -0.114*"ftp"'),
 (3,
  u'0.972*"db" + 0.149*"cs" + 0.089*"al" + 0.082*"cx" + 0.064*"bits" + -0.041*"file" + -0.021*"edu" + 0.021*"gas" + 0.020*"higher" + 0.020*"ah"'),
 (4,
  u'-0.675*"g9v" + 0.507*"0d" + 0.331*"_o" + -0.182*"b8f" + 0.129*"145" + 0.128*"6um" + -0.113*"1d9" + 0.110*"6ei" + 0.096*"3t" + 0.088*"75u"'),
 (5,
  u'0.244*"10" + 0.230*"14" + 0.225*"16" + 0.189*"12" + 0.186*"25" + 0.180*"15" + 0.178*"20" + 0.175*"13" + 0.174*"11" + 0.170*"00"'),
 (6,
  u'0.298*"file" + -0.233*"stephanopoulos" + -0.233*"mr" + -0.210*"know" + -0

Какая модель по вашему мнению лучше выделила темы?

Сложно сказать какой алгоритм лучше выделил темы. Так например, LSI лучше выделил тему про хоккей, а LDA про оружие